In [1]:
library(Seurat)
library(Signac)
library(chromVAR)
library(motifmatchr)
library(Matrix)
library(TFBSTools)
library(JASPAR2020)
library(BSgenome.Rnorvegicus.UCSC.rn6)
library(BiocParallel)
library(ggplot2)


Attaching SeuratObject


Attaching package: ‘Signac’


The following object is masked from ‘package:Seurat’:

    FoldChange





Attaching package: ‘TFBSTools’


The following object is masked from ‘package:Matrix’:

    Matrix


Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int

In [3]:

# snATAC data takes about 5-10 mins~ to read  from RDS
snatac_rds <- "harmony_with_predicted_id.rds"
rat_data <- readRDS(snatac_rds)

DefaultAssay(rat_data) <- "peaks"
Idents(rat_data) <- rat_data$predicted.id



In [4]:

# Takes 5 mins~
# LoadVertebrate Matrix
pfm <- getMatrixSet(x = JASPAR2020,
                   opts = list(collection = 'CORE', #collection to find
                   tax_group = 'vertebrates', #taxonomy group
                   all_versions = FALSE)
                   )


# Add Motif data
rat_data <- AddMotifs(
    object = rat_data,
    genome = BSgenome.Rnorvegicus.UCSC.rn6,
    pfm = pfm
)


Building motif matrix

Finding motif positions

Creating Motif object



In [5]:

# RUN CHROMVAR TAKES A WHILE 1:30 - 2 hours
register(MulticoreParam(48, progressbar = TRUE))
chromvar <- RunChromVAR(object = rat_data, genome = BSgenome.Rnorvegicus.UCSC.rn6)


Computing GC bias per region

Selecting background regions

Computing deviations from background



  |======================================================================| 100%



Constructing chromVAR assay



In [7]:

chromvar # Confirm chromvar assay present


An object of class Seurat 
314686 features across 81912 samples within 3 assays 
Active assay: peaks (291844 features, 290644 variable features)
 2 other assays present: RNA, chromvar
 3 dimensional reductions calculated: lsi, umap, harmony

In [167]:

# Perform differential testing 
cell_list <- c("Astrocytes", "ExNeuron", "InhNeuron", "Microglia", "Oligodendrocytes", "OPC")

cell_type <- cell_list[1] # CHANGE THIS FROM 1-6 TO PERFORM TEST ON EACH CELL TYPE

test_name <- "wilcox"

# EXAMPLE MARKERS for subset low high and celltype
differential.activity <- FindMarkers(chromvar, assay="chromvar", 
                                     ident.1 = 'cocaine.high',
                                     ident.2 = 'cocaine.low',
                                     group.by = 'condition',
                                     subset.ident = cell_type,
                                     test_use=test_name,
                                     only.pos = FALSE,
                                     logfc.threshold=0,
                                     min.pct = 0,
                                     mean.fxn = rowMeans,
                                     fc.name = "avg_diff"
                                    )

# FDR adjust pvalues
differential.activity$fdr = p.adjust(differential.activity$p_val, method='fdr')


# Write diff data to outpur tsv
diff_out <- sprintf("chromvar_diff_v2/%s_clusters_fdr.tsv", cell_type)
write.table(differential.activity, file=diff_out, sep="\t", row.names = TRUE, quote=FALSE)

differential.activity # Show output data

[1] "wilcox"


,p_val,avg_diff,pct.1,pct.2,p_val_adj,fdr
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MA0113.3,2.080285e-26,-0.3816178,0.773,0.832,1.551892e-23,1.551892e-23
MA0727.1,5.342531e-26,-0.3842041,0.784,0.837,3.985528e-23,1.992764e-23
MA0007.3,6.822374e-24,-0.3327425,0.665,0.747,5.089491e-21,1.696497e-21
MA0480.1,1.178422e-19,0.2447419,0.844,0.765,8.791027e-17,2.197757e-17
MA1150.1,5.922017e-17,0.3202596,0.995,0.993,4.417824e-14,8.835649e-15
MA0071.1,1.609840e-15,0.2488727,0.974,0.957,1.200940e-12,2.001567e-13
MA1151.1,1.151912e-13,0.2903658,0.996,0.993,8.593261e-11,1.227609e-11
MA0746.2,8.953663e-13,-0.1340090,0.116,0.185,6.679433e-10,8.349291e-11
MA0442.2,1.526744e-12,0.1942299,0.910,0.859,1.138951e-09,1.265501e-10
